In [34]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from geojson import Feature, FeatureCollection, Point
import folium

In [2]:
def create_polygon(country, lat_list, lon_list):
    '''
    Creates a polygon object
    Inputs:
        country (string)- Name to identify a row that will have a name (country), and a polygon
        lat_list(list) - List with latitude values for that polygon
        lon_list(list) - List with longitude values for that polygon
    Returns:
        Geopandas dataframe
    '''

    polygon_geom = Polygon(zip(lon_list, lat_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs='crs', geometry=[polygon_geom])       
    #polygon.plot()
    polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
    polygon['country'] = country
    return polygon

In [3]:
def map_polygon(country, polygon):
    
    '''
    Creates a folium map with the geopandas dataframe that contains a polygon object
    Inputs:
        country(string): Country code to center the map. Valid values: "MEX", "USA",
        "BRA", "IND"
        polygon: geopandas dataframe coming from  "create_polygon function"
    returns:
        folium map
      '''
    
    if country == "MEX":
        centroid = [30.422055, -111.822862]
    elif country == "USA":
        centroid = [39.8282, -98.5795]
    elif country == "BRA":
        centroid = [-14.235004,-51.92528]
    elif country == "IND":
        centroid = [21,78]
    elif country == "ARG":
        centroid = [-38.416097,-63.616672]
    else:
        centroid = [-0.178992, 106.684306]
    m = folium.Map(centroid, zoom_start=3, tiles='cartodbpositron')
    folium.GeoJson(polygon).add_to(m)
    folium.LatLngPopup().add_to(m)
    return m

In [4]:
vertices = {
    "Mexico": [[  32.5089,17.4693, 14.5926,  17.8009, 21.5987, 25.9440,29.7809,  31.7837,32.7191],
               [-117.688,-108.8755, -92.2633, -86.4835,-86.3641, -97.1734,-101.4955, -106.5301, -114.7379]],
    "USA": [ [49.1280,32.3172, 32.6117,  30.6830, 28.9630, 24.88,  23.9716, 49.8366, 45.1823, 49.2071 ],
            [ -127.2017,-123.4697, -116.8360, -108.3712, -102.89, -96.4290, -80.5781,  -59.7335, -79.1901,  -94.6183]],
    "Brazil": [[ 4.9398, -7.8945, -33.2312,-29.7212, -18.2097, -8.4038 , 1.7537, 4.3357, 5.2997, 2.4527],
              [-49.0227, -27.9692, -52.3317, -57.2958, -58.1720, -74.9689, -70.1870, -64.6713, -60.5858, -55.9057]],
    "India": [[ 33.5206, 24.1218,7.0992, 21.8384,  26.2269,  28.2497],
             [76.8572, 66.4419, 77.3348, 89.6500, 89.9424, 88.2061,  88.9558]],
    "Argentina":[[-57.3993,-52.9911, -49.6819,  -21.9997, -21.3698, -25.2360, -25.6445, -27.2274,  -33.9466, -36.7613],
                 [-62.7675, -70.8459, -74.1001, -68.6180, -62.9491, -57.7228, -53.5515, -53.6808, -58.5417,  -55.7967]],
    "Indonesia": [[3.3380, 7.1744,  1.1003, 1.9544,0.9118, 4.4822, 5.0909, -1.8239, -10.1594, -11.5231 ],
                  [94.0077,95.3667, 103.7839, 109.6260, 110.7338,  115.9323, 127.5464,141.0061, 140.9391,98.1938]]
            }

In [5]:
m = folium.Map([6.1750, 106.8283], zoom_start=3, tiles='cartodbpositron')
folium.LatLngPopup().add_to(m)

m


In [6]:
# lats_ARG = [-57.3993,-52.9911, -49.6819,  -21.9997, -21.3698, -25.2360, -25.6445, -27.2274,  -33.9466, -36.7613]
# lons_ARG = [-62.7675, -70.8459, -74.1001, -68.6180, -62.9491, -57.7228, -53.5515, -53.6808, -58.5417,  -55.7967]
# lats_INDO = [3.3380, 7.1744,  1.1003, 1.9544,0.9118, 4.4822, 5.0909, -1.8239, -10.1594, -11.5231 ]
# lons_INDO = [94.0077,95.3667, 103.7839, 109.6260, 110.7338,  115.9323, 127.5464,141.0061, 140.9391,98.1938]

In [7]:
map_polygon("ARG",create_polygon("ARG", vertices['Argentina'][0], vertices['Argentina'][1]))

In [8]:
map_polygon("INDO",create_polygon("INDO", vertices['Indonesia'][0], vertices['Indonesia'][1]))

In [9]:
map_polygon("MEX",create_polygon("Mx", vertices['Mexico'][0], vertices['Mexico'][1]))

In [10]:
map_polygon("BRA",create_polygon("Bra", vertices['Brazil'][0], vertices['Brazil'][1]))

In [11]:
map_polygon("IND",create_polygon("IND", vertices['India'][0], vertices['India'][1]))

In [71]:
def create_polygon_files(dict_lat_longs):
    '''
    Creates a csv with the polygon information
    Input:
        dict_lat_longs: Dictionary which keys are country codes, and values a list of lists with lat long values
    Returns:
        Pandas dataframe, and creates a csv file'''
    ls_gdfs = []
    for key, val in dict_lat_longs.items():
        gdf = create_polygon(key, val[0], val[1])
        df = pd.DataFrame(gdf)
        ls_gdfs.append(df)
    geo_polygons = gpd.GeoDataFrame( pd.concat( ls_gdfs, ignore_index=True) )
    geo_polygons.to_file('polygons.json', driver='GeoJSON')
    
    return geo_polygons
        

In [72]:
create_polygon_files(vertices)

,geometry,country
0,"POLYGON ((-117.68800 32.50890, -108.87550 17.4...",Mexico
1,"POLYGON ((-127.20170 49.12800, -123.46970 32.3...",USA
2,"POLYGON ((-49.02270 4.93980, -27.96920 -7.8945...",Brazil
3,"POLYGON ((76.85720 33.52060, 66.44190 24.12180...",India
4,"POLYGON ((-62.76750 -57.39930, -70.84590 -52.9...",Argentina
5,"POLYGON ((94.00770 3.33800, 95.36670 7.17440, ...",Indonesia
